In [1]:
import pandas as pd
# load data
df = pd.read_csv('../data/processed/merged_data.csv')

In [2]:
df.columns

Index(['course_id', 'course_type_broad', 'unit_id', 'unit_level_code',
       'unit_level_name', 'unit_foe_detailed', 'unit_foe_narrow', 'foe_code',
       'unit_foe_broad', 'eftsl_2024', 'funding_nation', 'funding_type',
       'overload', 'funding_cluster', 'max_student_contrib_2024',
       'commonwealth_contrib_2024', 'max_student_contrib_gf_2024',
       'commonwealth_contrib_gf_2024', 'is_funding_cluster_variable',
       'special_course_code', 'max_contrib_indicator', 'foe_detailed_title',
       'foe_detailed', 'foe_narrow', 'foe_broad', 'foe_error', 'special_code',
       'CSP_gov_payment'],
      dtype='object')

#### Data preparation

In [ ]:
# Select relevant features for modeling
features_to_use = [
    'course_id', 'course_type_broad', 'unit_id', 'unit_level_code',
       'unit_level_name', 'unit_foe_detailed', 'unit_foe_narrow', 'foe_code',
       'unit_foe_broad', 'eftsl_2024', 'funding_nation', 'funding_type',
       'overload', 'funding_cluster', 'max_student_contrib_2024',
       'commonwealth_contrib_2024', 'max_student_contrib_gf_2024',
       'commonwealth_contrib_gf_2024', 'is_funding_cluster_variable',
       'special_course_code', 'max_contrib_indicator', 'foe_detailed_title',
       'foe_detailed','special_code',
]
categorical_features = ['sex_ed', 'ethnicity', 'mode_of_arrival', 'primary_diagnosis_ICD10AM_chapter', 'arrival_day_of_week']
numerical_features = ['age_ed', 'triage_category', 'mental_health_attendance', 'arrival_hour', 'is_weekend']

# Create a preprocessing pipeline for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Keep numerical features as they are
)

In [ ]:
# Define final features and target
X = merged_df[features_to_use]
y = merged_df['was_admitted']

In [ ]:
print("\n--- Corrected Data Splitting (70% Train, 15% Validation, 15% Test) ---")
# First split: 70% train, 30% temp (val + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Second split: Split the 30% temp set into 50% validation and 50% test (which is 15% of the original data each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")


--- Corrected Data Splitting (70% Train, 15% Validation, 15% Test) ---
Training set size: 363572
Validation set size: 77908
Test set size: 77909


In [ ]:
print("\n---Handling Class Imbalance ---")
counts = y_train.value_counts()
scale_pos_weight_value = counts[0] / counts[1]
print(f"Calculated scale_pos_weight for XGBoost: {scale_pos_weight_value:.2f}")


---Handling Class Imbalance ---
Calculated scale_pos_weight for XGBoost: 4.26


In [ ]:
# --- Helper function for evaluation ---
def evaluate_models(models, X_train, y_train, X_test, y_test):
    """A helper function to train and evaluate a dictionary of models."""
    results_data = []
    for name, model in models.items():
        print(f"  Training and evaluating {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        auc = roc_auc_score(y_test, y_proba)
        results_data.append([name, accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), 
                               recall_score(y_test, y_pred), f1_score(y_test, y_pred), auc])
    
    return pd.DataFrame(results_data, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']).sort_values('ROC AUC', ascending=False)

#### Run Model

In [ ]:
import xgboost as xgb

print("\n--- Evaluating 3 models using ALL features ---")

preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'
)

# Define the 3 models
lr = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000))])
rf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))])
xgb = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight_value, eval_metric='logloss'))])

models_features = {
    "Logistic Regression": lr,
    "Random Forest": rf,
    "XGBoost": xgb
}


--- Evaluating 3 models using ALL features ---


In [ ]:
results_all_features = evaluate_models(models_features, X_train, y_train, X_test, y_test)
print("\n--- Performance on train data with ALL features ---")
print(results_all_features)

  Training and evaluating Logistic Regression...
  Training and evaluating Random Forest...
  Training and evaluating XGBoost...

--- Performance on train data with ALL features ---
                 Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
0  Logistic Regression  0.951251   0.815065  0.962089  0.882495  0.994975
2              XGBoost  0.946065   0.785968  0.984687  0.874177  0.994967
1        Random Forest  0.967603   0.947208  0.878710  0.911674  0.994868


#### Feature Selection

In [ ]:
import shap
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

print("\n---  Selecting best features using SHAP ---")

lgbm = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', lgb.LGBMClassifier(random_state=42, scale_pos_weight=scale_pos_weight_value,verbose=-1))])

lgbm.fit(X_train, y_train)

# --- SHAP Value Calculation ---
# 1. We need to get the actual classifier model from the pipeline
lgbm_classifier = lgbm.named_steps['classifier']

# 2. We need the preprocessed data that the classifier was trained on.
# We will use the validation set (X_val) for calculating SHAP values.
# This is a good practice as it's a representative sample and faster than using the full training set.
preprocessor = lgbm.named_steps['preprocessor']
X_val_transformed = preprocessor.transform(X_val)
# The output is a sparse matrix, convert it to a dense numpy array for SHAP
X_val_transformed_dense = X_val_transformed.toarray()
# Get the feature names from the preprocessor
feature_names_raw = preprocessor.get_feature_names_out()


# 3. Create a SHAP explainer and calculate SHAP values
print("  Calculating SHAP values... (This may take a moment)")
explainer = shap.TreeExplainer(lgbm_classifier)
# shap_values is a list of two arrays for binary classification (one for each class)
shap_values = explainer.shap_values(X_val_transformed_dense)

# 4. Calculate global feature importance from SHAP values
# We take the mean absolute SHAP value for each feature for the positive class (class 1)
shap_importance = np.mean(np.abs(shap_values[1]), axis=0)
importance_df = pd.DataFrame({
    'feature': feature_names_raw, 
    'shap_importance': shap_importance
}).sort_values('shap_importance', ascending=False)

print("  SHAP calculation complete.")

# --- Feature Selection based on SHAP importance ---
# Select features with a mean absolute SHAP value greater than a small threshold
selected_features_raw = importance_df[importance_df['shap_importance'] > 0.01]['feature'].tolist()

# Map these raw encoded feature names back to the original columns (same logic as before)
selected_features = []
for f in selected_features_raw:
    if f.startswith('remainder__'):
        selected_features.append(f.replace('remainder__', ''))
    elif f.startswith('cat__'):
        col = f.replace('cat__', '').split('_')[0]
        selected_features.append(col)
    else:
        selected_features.append(f)

# Remove duplicates
selected_features = list(dict.fromkeys([f for f in selected_features if f in X_train.columns]))
print(f"Selected {len(selected_features)} original features based on SHAP: {selected_features}")

# Create new dataframes for train, validation, and test sets with only the selected features
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]
print(f"Created new data splits with {len(selected_features)} features.")


---  Selecting best features using SHAP ---
  Calculating SHAP values... (This may take a moment)
  SHAP calculation complete.
Selected 6 original features based on SHAP: ['age_ed', 'triage_category', 'mental_health_attendance', 'arrival_hour', 'ethnicity', 'is_weekend']
Created new data splits with 6 features.


/opt/anaconda3/envs/ml/lib/python3.12/site-packages/shap/explainers/_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


In [ ]:
import xgboost as xgb

# Identify which of the selected features are categorical for the new preprocessor
selected_categorical = [f for f in selected_features if f in categorical_features]

# Create a new preprocessor for the selected feature subset
preprocessor_selected = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), selected_categorical)],
    remainder='passthrough'
)

# Define the 3 models
lr_selected = Pipeline(steps=[('preprocessor', preprocessor_selected), ('classifier', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000))])
rf_selected = Pipeline(steps=[('preprocessor', preprocessor_selected), ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))])
xgb_selected = Pipeline(steps=[('preprocessor', preprocessor_selected), ('classifier', xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight_value, eval_metric='logloss'))])

models_selected_features = {
    "Logistic Regression": lr_selected, 
    "Random Forest": rf_selected, 
    "XGBoost": xgb_selected
}

In [ ]:
# train
results_select_features = evaluate_models(models_selected_features, X_train_selected, y_train, X_test_selected, y_test)
print("\n--- Performance on train data with select features ---")
print(results_select_features)

  Training and evaluating Logistic Regression...
  Training and evaluating Random Forest...
  Training and evaluating XGBoost...

--- Performance on train data with select features ---
                 Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
0  Logistic Regression  0.951251   0.814310  0.963505  0.882647  0.994978
2              XGBoost  0.944641   0.778231  0.991635  0.872067  0.994807
1        Random Forest  0.955114   0.842806  0.939288  0.888435  0.991810


#### Hyperparameter Tuning

In [ ]:
print("\n---Hyperparameter Tuning on Selected Features ---")


# --- Tune Random Forest ---
print("  Tuning Random Forest...")
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor_selected), ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))])
rf_param_dist = {'classifier__n_estimators': [100, 200], 'classifier__max_depth': [5, 10, None], 'classifier__min_samples_leaf': [1, 2, 4]}
rf_rand_search = RandomizedSearchCV(rf_pipeline, rf_param_dist, n_iter=5, cv=3, scoring='roc_auc', random_state=42, n_jobs=-1, verbose=1)
rf_rand_search.fit(X_val_selected, y_val)
rf_tuned_pipeline = rf_rand_search.best_estimator_
print(f"Best RF Params: {rf_rand_search.best_params_}")


# --- Tune XGBoost ---
print("\n  Tuning XGBoost...")
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor_selected), ('classifier', xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight_value,  eval_metric='logloss'))])
xgb_param_dist = {'classifier__n_estimators': [100, 200], 'classifier__max_depth': [3, 5, 7], 'classifier__learning_rate': [0.01, 0.05, 0.1]}
xgb_rand_search = RandomizedSearchCV(xgb_pipeline, xgb_param_dist, n_iter=5, cv=3, scoring='roc_auc', random_state=42, n_jobs=-1, verbose=1)
xgb_rand_search.fit(X_val_selected, y_val)
xgb_tuned_pipeline = xgb_rand_search.best_estimator_
print(f"Best XGBoost Params: {xgb_rand_search.best_params_}")


---Hyperparameter Tuning on Selected Features ---
  Tuning Random Forest...
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best RF Params: {'classifier__n_estimators': 100, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 5}

  Tuning XGBoost...
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best XGBoost Params: {'classifier__n_estimators': 100, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.01}


In [ ]:
# Create a dictionary of the newly tuned models
tuned_models = {
    "Random Forest (Tuned)": rf_tuned_pipeline,
    "XGBoost (Tuned)": xgb_tuned_pipeline
}

In [ ]:
results_tuned = evaluate_models(tuned_models, X_val_selected, y_val, X_test_selected, y_test)

print("\n--- Performance of Tuned Models on Test Set ---")
print(results_tuned)

  Training and evaluating Random Forest (Tuned)...
  Training and evaluating XGBoost (Tuned)...

--- Performance of Tuned Models on Test Set ---
                   Model  Accuracy  Precision  Recall  F1 Score   ROC AUC
1        XGBoost (Tuned)  0.943306   0.770438     1.0  0.870336  0.994956
0  Random Forest (Tuned)  0.943306   0.770438     1.0  0.870336  0.994933


#### Model Evaluation & Comparison

In [ ]:
print("\n--- Final Model Selection ---")
# Concatenate results for a final comparison
# This part of your code is correct.
final_comparison = pd.concat([results_all_features, results_select_features, results_tuned]).reset_index(drop=True)
print("\n--- Overall Performance Summary ---")
print(final_comparison)

best_model_details = final_comparison.sort_values('ROC AUC', ascending=False).iloc[0]
best_model_name = best_model_details['Model']

print(f"\n🏆 The best model is '{best_model_name}', with an ROC AUC of {best_model_details['ROC AUC']:.4f}.")


--- Final Model Selection ---

--- Overall Performance Summary ---
                   Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
0    Logistic Regression  0.951251   0.815065  0.962089  0.882495  0.994975
1                XGBoost  0.946065   0.785968  0.984687  0.874177  0.994967
2          Random Forest  0.967603   0.947208  0.878710  0.911674  0.994868
3    Logistic Regression  0.951251   0.814310  0.963505  0.882647  0.994978
4                XGBoost  0.944641   0.778231  0.991635  0.872067  0.994807
5          Random Forest  0.955114   0.842806  0.939288  0.888435  0.991810
6        XGBoost (Tuned)  0.943306   0.770438  1.000000  0.870336  0.994956
7  Random Forest (Tuned)  0.943306   0.770438  1.000000  0.870336  0.994933

🏆 The best model is 'Logistic Regression', with an ROC AUC of 0.9950.
